## Documentation Revision Agent

This script automates the improvement of technical documentation by applying AI-powered revisions based on quality analysis reports. It focuses on enhancing clarity and readability while preserving technical accuracy.

### Technologies Used

1. **Google Gemini API** (`gemini-1.5-flash`)
   - Processes complex revision requests
   - Maintains context-aware document structure
   - Handles large documents (up to 8K tokens)

2. **Hybrid Revision Approach**
   - Regex-based simple replacements
   - AI-powered structural improvements
   - Rule-based content preservation

3. **Core Python Modules**
   - `re`: Pattern matching for text transformations
   - `json`: Processes analysis reports
   - `os`: File system operations
   - `time`: API rate limiting

### Why This Approach?

The implementation focuses on:
- **Precision Preservation**:
  - Code blocks remain untouched
  - Frontmatter/YAML metadata protected
  - Original structure maintained
- **Efficiency**:
  - Simple replacements avoid API calls
  - Complex changes handled by AI
  - 1-second rate limiting for API
- **Safety**:
  - Strict revision boundaries
  - No new content added
  - Fallback to original on errors
- **Actionability**:
  - Focuses on readability/style improvements
  - Processes suggestions from analysis reports
  - Outputs ready-to-use Markdown files

### Revision Workflow

1. **Suggestion Extraction**:
   - Reads analysis reports (JSON)
   - Filters readability/style suggestions
   - Combines suggestions from multiple categories

2. **Simple Replacements**:
   - Direct text substitutions
   - Jargon clarification patterns
   - Before/after phrase replacements

3. **Complex Revisions**:
   - Structural improvements
   - Sentence rephrasing
   - Flow optimization
   - Compliance with style guides

4. **Output Generation**:
   - Creates new Markdown files
   - Maintains directory structure
   - Preserves all original non-text elements

In [1]:
import os
import re
import json
import google.generativeai as genai
import time


GEMINI_API_KEY = "Secret..."  # Replace with your actual API key
genai.configure(api_key=GEMINI_API_KEY)

# I used an efficient model for free tier
GEMINI_MODEL = "gemini-1.5-flash"

class DocumentationRevisionAgent:
    def __init__(self, model=GEMINI_MODEL):
        self.model = genai.GenerativeModel(model)
        self.last_call_time = 0
    
    def _rate_limit(self):
        """Enforce API rate limiting (1 request/second)"""
        elapsed = time.time() - self.last_call_time
        if elapsed < 1.0:
            time.sleep(1.0 - elapsed)
        self.last_call_time = time.time()
    
    def _apply_simple_revisions(self, content, suggestions):
        """Apply straightforward text replacements"""
        revised = content
        simple_replacements = []
        
        for suggestion in suggestions:
            # Process simplification suggestions 
            if "->" in suggestion or "Change" in suggestion or "replace" in suggestion.lower():
                # To Match Patterns
                match = re.search(r"['\"](.*?)['\"]\s*(?:to|->|with)\s*['\"](.*?)['\"]", suggestion)
                if match:
                    old_phrase, new_phrase = match.groups()
                    simple_replacements.append((old_phrase, new_phrase))
            
            # Process jargon replacements
            elif "jargon" in suggestion.lower() or "term" in suggestion.lower():
                # Match patterns like: Simplify term 'API'
                match = re.search(r"term\s+['\"](.*?)['\"]", suggestion)
                if match:
                    jargon = match.group(1)
                    simple_replacements.append((jargon, f"{jargon} (technical term)"))
        
        # Apply all replacements
        for old, new in simple_replacements:
            revised = revised.replace(old, new)
        
        return revised
    
    def _apply_complex_revisions(self, content, suggestions):
        """Use Gemini for complex structural/sentence revisions"""
        self._rate_limit()
        try:
            # Create prompt with clear instructions
            prompt = f"""Revise the documentation content based on these suggestions:
            {json.dumps(suggestions, indent=2)}
            
            IMPORTANT RULES:
            1. PRESERVE all code blocks exactly as-is (do not modify content between ``` ```)
            2. MAINTAIN original structure and headings
            3. ONLY change text in sections mentioned in suggestions
            4. KEEP YAML frontmatter (content between ---) completely unchanged
            5. DO NOT add any new sections or content not in the original
            6. FOCUS on clarity and readability improvements
            
            Original Content:
            {content}
            """
            
            response = self.model.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=0.3,
                    max_output_tokens=8000  # Allow for longer documents
                )
            )
            return response.text.strip()
        except Exception as e:
            print(f"Revision error: {str(e)}")
            return content
    
    def revise_document(self, original_md_path, report_path, output_path):
        """Revise documentation based on analysis suggestions"""
        print(f"\nStarting revision for: {os.path.basename(original_md_path)}")
        
        # Load original content
        try:
            with open(original_md_path, 'r', encoding='utf-8') as f:
                original_content = f.read()
        except Exception as e:
            print(f"Error reading original file: {str(e)}")
            return
        
        # Load analysis report
        try:
            with open(report_path, 'r', encoding='utf-8') as f:
                report = json.load(f)
        except Exception as e:
            print(f"Error reading report file: {str(e)}")
            return
        
        # Extract suggestions from readability and style categories
        suggestions = []
        for category in ['readability', 'style']:
            if 'results' in report and category in report['results']:
                cat_suggestions = report['results'][category].get('suggestions', [])
                print(f"Found {len(cat_suggestions)} {category} suggestions")
                suggestions.extend(cat_suggestions)
        
        if not suggestions:
            print("No suggestions to apply")
            return
        
        print(f"Total suggestions to process: {len(suggestions)}")
        
        # Apply simple revisions first
        revised_content = self._apply_simple_revisions(original_content, suggestions)
        
        # Apply complex revisions
        print("Applying complex revisions with Gemini...")
        revised_content = self._apply_complex_revisions(revised_content, suggestions)
        
        # Save revised document to new file
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(revised_content)
        
        print(f"Revised document saved to: {output_path}")
        return revised_content

def main():
    """Main function for document revision"""
    reviser = DocumentationRevisionAgent()
    
    print("MoEngage Documentation Revision Agent")
    print("=" * 50)
    print("This tool revises documentation based on analysis suggestions")
    print("Focuses on readability and style improvements\n")
    
    file_sets = [
        {
            'original': "Scraped_Docs/web_sdk_overview.md",
            'report': "Outputs_Task1/Web_Sdk_Overview_Analysis.json",
            'output': "Outputs_Task2/web_sdk_overview_revised.md"
        },
        {
            'original': "Scraped_Docs/getting_started_with_react_native_sdk.md",
            'report': "Outputs_Task1/React_Native_Sdk_Analysis.json",
            'output': "Outputs_Task2/getting_started_with_react_native_sdk_revised.md"
        }
    ]
    
    # Process both files
    for i, files in enumerate(file_sets, 1):
        print(f"\nProcessing File #{i}")
        print("-" * 40)
        print(f"Original: {files['original']}")
        print(f"Report: {files['report']}")
        print(f"Output: {files['output']}")
        
        # Run revision for this file
        reviser.revise_document(
            files['original'],
            files['report'],
            files['output']
        )
    
    print("\nRevision process complete for both files!")

if __name__ == "__main__":
    main()

MoEngage Documentation Revision Agent
This tool revises documentation based on analysis suggestions
Focuses on readability and style improvements


Processing File #1
----------------------------------------
Original: Scraped_Docs/web_sdk_overview.md
Report: Outputs_Task1/Web_Sdk_Overview_Analysis.json
Output: Outputs_Task2/web_sdk_overview_revised.md

Starting revision for: web_sdk_overview.md
Found 2 readability suggestions
Found 1 style suggestions
Total suggestions to process: 3
Applying complex revisions with Gemini...
Revised document saved to: Outputs_Task2/web_sdk_overview_revised.md

Processing File #2
----------------------------------------
Original: Scraped_Docs/getting_started_with_react_native_sdk.md
Report: Outputs_Task1/React_Native_Sdk_Analysis.json
Output: Outputs_Task2/getting_started_with_react_native_sdk_revised.md

Starting revision for: getting_started_with_react_native_sdk.md
Found 2 readability suggestions
Found 4 style suggestions
Total suggestions to process: